# UDH Log Analysis

In [2]:
%matplotlib inline

import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
import json
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
pd.set_option('display.height', 10)
pd.set_option('display.max_rows', 10)

# Our modules
#import splunk_api

height has been deprecated.



In [3]:
def remove_digits(s):
    return ''.join([i for i in s if not i.isdigit()])

def remove_session_ids(s):
    return re.sub(r'[\w]{8}-[\w]{4}-[\w]{4}-[\w]{4}-[\w]{12}', '', s)

def generalise_text(s):
    return ''.join([("D" if i.isdigit() else i) for i in s])

In [4]:
def frequency_count(items):
    result = dict()
    for item in items:
        result[item] = result.get(item, 0) + 1
    return result

In [5]:
filename = "../data/uhd_bigquery_results-20180216-131501.json"
filename = "../data/uhd_bigquery_results-20180216-145011.json" #level=ERROR
filename = "../data/error_logs.json" #level=ERROR

In [ ]:
# log_dir = '/home/dreijer/logs/log-dump-qluhd/logDump/'
# filenames = []
# for file in os.listdir(log_dir):
#     if file.endswith('.json'):
#         filenames.append(os.path.join(log_dir, file))

# logs_df = pd.DataFrame()
# for filename in filenames:
#     temp_logs_df = pd.DataFrame([json.loads(line) for line in open(filename)])
#     #temp_logs_df['datetime'] = pd.to_datetime(temp_logs_df['timestamp'], unit='s')
#     temp_logs_df = temp_logs_df[temp_logs_df['logLevel'] == 'ERROR']
#     logs_df = logs_df.append(temp_logs_df).reset_index(drop=True)
#     print('Processed ', filename)
# logs_df

In [8]:
logs_df = pd.DataFrame([json.loads(line) for line in open(filename)])
logs_df

,buildconfig,customerID,deviceID,fingerprint,logEvent,logLevel,tag,timestamp,version
0,Release,100407731,943BB16960AA,Quickline/quickline_dciw387/dciw387:7.1.1/NYJ8...,buildFromJson(): Error parsing channelInternal...,ERROR,ChannelInternalProviderData,2018-02-27 11:56:58 UTC,1.8.0-stb
1,Release,100407731,943BB16960AA,Quickline/quickline_dciw387/dciw387:7.1.1/NYJ8...,setResult(): failed. Future already finished.,ERROR,Future,2018-02-27 11:55:11 UTC,1.8.0-stb
2,Release,100407731,943BB16960AA,Quickline/quickline_dciw387/dciw387:7.1.1/NYJ8...,[CfgM.Configuration ] Access to unknown prope...,ERROR,DvbServerLogReceiver$DvbServerLogReceiverRunnable,2018-02-27 11:55:06 UTC,1.8.0-stb
3,Release,100407731,943BB16960AA,Quickline/quickline_dciw387/dciw387:7.1.1/NYJ8...,setResult(): failed. Future already finished.,ERROR,Future,2018-02-27 11:39:50 UTC,1.8.0-stb
4,Release,100407731,943BB16960AA,Quickline/quickline_dciw387/dciw387:7.1.1/NYJ8...,setResult(): failed. Future already finished.,ERROR,Future,2018-02-27 11:39:34 UTC,1.8.0-stb
...,...,...,...,...,...,...,...,...,...
360804,Release,Unknown,Unknown,Quickline/quickline_dciw387/dciw387:7.1.1/NYJ7...,"E 00:00:00,284 [Osal.File ] InarisO...",ERROR,MiddlewareLogReceiverThread$MiddlewareLogRecei...,2017-01-01 00:00:37 UTC,1.7.0-stb
360805,Release,Unknown,Unknown,Quickline/quickline_dciw387/dciw387:7.1.1/NYJ7...,log(): [SingleSignOn] Error: java.lang.Throwab...,ERROR,ErrorHandler,2017-01-01 00:00:34 UTC,1.7.0-stb
360806,Release,Unknown,Unknown,Quickline/quickline_dciw387/dciw387:7.1.1/NYJ7...,loadSettingFromQmcProxy(): com.android.volley....,ERROR,QmcProxy,2017-01-01 00:00:34 UTC,1.7.0-stb
360807,Release,Unknown,Unknown,Quickline/quickline_dciw387/dciw387:7.1.1/NYJ7...,loadSubscriberToken(): java.lang.NullPointerEx...,ERROR,TokenLoader,2017-01-01 00:00:34 UTC,1.7.0-stb


In [10]:
#logs_df['datetime'] = pd.to_datetime(logs_df['timestamp'], unit='s')

In [11]:
#logs_df = logs_df[logs_df['deviceID'] == '943BB164FCF3'].reset_index(drop=True)

In [12]:
logs_df['datetime'].min()

KeyError: 'datetime'

In [ ]:
logs_df['datetime'].max()

ruediuhd                        102586
100407731                        70533
uhdtestwwz                       35665
ursina-atv                       26623
100344760                        21981
100406068                        12915
100405714                        10843
100405044                         9893
romanuhd                          7574
uhdtestlukas                      5399
100405046                         5036
100407720                         4268
100405041                         4073
uhdralf                           3859
mathias1                          3698
Unknown                           3178
ottmysports                       2458
100407723                         2303
100390547                         2248
100407729                         2186
100405039                         2163
100366559                         2103
yann.steulet                      1773
stefan-lyss                       1619
nadineuhd                         1471
veliuhd                  

In [11]:
def generate_mac_dates(logs_df):
    mac_dates_df = pd.DataFrame(columns=['mac', 'timespan_start', 'timespan_end'])

    for mac in logs_df['customerID'].unique():

        if mac == 'Unknown':
            continue

        date_min = logs_df[logs_df['customerID'] == mac]['datetime'].min()
        date_max = logs_df[logs_df['customerID'] == mac]['datetime'].max()

        data_point = [mac, date_min, date_max]
        mac_dates_df.loc[len(mac_dates_df)] = data_point
        
    mac_dates_df.to_csv('mac_dates.csv')

In [12]:
generate_mac_dates(logs_df)

In [14]:
with pd.option_context('display.max_rows', 100):
    print(logs_df["deviceID"].value_counts())

943BB1000134    102579
943BB16960AA     70253
943BB1FFFE28     42743
943BB164FCF3     26628
943BB1FFFFFF     24946
943BB1664D26     14934
943BB16810AA     14700
943BB1656BA8     12624
943BB1FFFFF8      7357
943BB162C13D      5396
943BB1FFFE08      5022
Unknown           4492
943BB16910AA      4267
943BB1FFFFF0      4069
943BB16930AA      2302
943BB16950AA      2182
943BB1A4DB99      2103
943BB1FFFFE8      1965
943BB1FFFFE0      1514
943BB1FFFE00      1251
3ACFC5000020      1207
943BB164D94D      1037
943BB1000181       929
943BB1FFFFD0       671
943BB1FFFFD8       588
943BB16920AA       556
943BB1A57FBE       473
943BB16940AA       401
943BB16830AA       400
943BB1668017       394
943BB16900AA       358
943BB1000252       329
943BB16840AA       277
943BB162F2D8       263
943BB1000176       247
943BB1639745       241
943BB16850AA       173
943BB1FFFE10       158
943BB1654BD7       140
943BB16820AA       109
3ACFC5000008       107
3ACFC5000018        97
943BB162A17F        92
943BB1669DE

In [14]:
counts = frequency_count(logs_df["logEvent"].values)
counts_df = pd.DataFrame(sorted([(c,line) for line, c in counts.items()], reverse=True), columns=["count", "logEvent"])
counts_df

,count,logEvent
0,905,onError(): com.android.volley.NoConnectionErro...
1,680,setResult(result): failed. Future already fini...
2,640,buildFromJson(): Error parsing channelInternal...
3,328,Request: GetChannelsRequest
4,135,onError(): com.android.volley.NoConnectionErro...
5,88,[SM.UrlParser ] Parsing DVB URL was not...
6,82,[SM.UrlParser ] Error at location: .0\n
7,69,[CfgM.Configuration ] Access to unknown prope...
8,48,setResult(): failed. Future already finished.
9,37,setFailure(): failed. Future already finished.


In [15]:
# line level n gram
 

## 1. Text preprocessing

In [16]:
corpus = list(logs_df['logEvent'])

# Remove all session IDs (in format 12345678-1234-1234-ab12cd34ef56)
corpus = [remove_session_ids(sentence) for sentence in corpus]

# Remove all digits
corpus = [remove_digits(sentence) for sentence in corpus]

## 2. Feature extraction

In [17]:
vectorizer = TfidfVectorizer(stop_words=None, analyzer='word', ngram_range=(1,1)) #ngram
X = vectorizer.fit_transform(corpus)
vectorizer.vocabulary_

{u'abort': 0,
 u'access': 1,
 u'activitythread': 2,
 u'addcallback': 3,
 u'address': 4,
 u'already': 5,
 u'android': 6,
 u'anmeldung': 7,
 u'api': 8,
 u'app': 9,
 u'appe': 10,
 u'associated': 11,
 u'at': 12,
 u'attempt': 13,
 u'authorized': 14,
 u'availability': 15,
 u'available': 16,
 u'awaitcountdownlatch': 17,
 u'awaitseconds': 18,
 u'backend': 19,
 u'behindlivewindowexception': 20,
 u'benutzername': 21,
 u'buildfromjson': 22,
 u'but': 23,
 u'by': 24,
 u'cablespectruminversion': 25,
 u'call': 26,
 u'called': 27,
 u'caused': 28,
 u'cfgm': 29,
 u'ch': 30,
 u'channel': 31,
 u'channelinternalproviderdata': 32,
 u'channelmanager': 33,
 u'channelmanagerimpl': 34,
 u'channelprogramupdater': 35,
 u'channelprogramupdaterrunnable': 36,
 u'channelprogramupdaterthread': 37,
 u'channels': 38,
 u'channelsloader': 39,
 u'channelsloaderhandlerthread': 40,
 u'channeltuner': 41,
 u'clienterror': 42,
 u'code': 43,
 u'com': 44,
 u'configuration': 45,
 u'connect': 46,
 u'connectexception': 47,
 u'connec

In [18]:
X.todense()

matrix([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ..., 
        [ 0.        ,  0.        ,  0.        , ...,  0.37199413,
          0.        ,  0.        ],
        [ 0.        ,  0.35328053,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.37056437,
          0.        ,  0.        ]])

## 3. Clustering

In [19]:
max_silhouette_avg = -1
optimal_k = 1
for k in range(2, 21):
    
    km = KMeans(n_clusters=k, max_iter=100)
    #print("Clustering sparse data with %s" % km)
    labels = km.fit_predict(X)
    
    silhouette_avg = silhouette_score(X, labels)
    print(k, silhouette_avg)
    
    if silhouette_avg > max_silhouette_avg:
        max_silhouette_avg = silhouette_avg
        optimal_k = k

(2, 0.35408065686792206)
(3, 0.50448998495560138)
(4, 0.64685635721901313)
(5, 0.75255953096314387)
(6, 0.81410495894770196)
(7, 0.85503770428799319)
(8, 0.87211690537621045)
(9, 0.89016585932440773)
(10, 0.90145311694769803)
(11, 0.9083038532651786)
(12, 0.91636716152364306)
(13, 0.92481653967207544)
(14, 0.92838192542490661)
(15, 0.9321475179131663)
(16, 0.94866425719570513)
(17, 0.95131567842312337)
(18, 0.95523481947598243)
(19, 0.95709002161519752)
(20, 0.95974683510248171)


In [20]:
true_k = optimal_k
km = KMeans(n_clusters=true_k, max_iter=100)

print("Clustering sparse data with %s" % km)
labels = km.fit_predict(X)

Clustering sparse data with KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=20, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)


In [21]:
unique_elements, counts_elements = np.unique(labels, return_counts=True)
print(counts_elements)

[728 905 695 645 222 328  82  41  88  69  37  15  23  25   9  37  23  57
  26 135]


In [22]:
counts = frequency_count(corpus)
counts_df = pd.DataFrame(sorted([(c, line) for line, c in counts.items()], reverse=True), columns=["count", "logEvent"])
counts_df

,count,logEvent
0,905,onError(): com.android.volley.NoConnectionErro...
1,695,[AppE ] Process sleep took ms\n
2,680,setResult(result): failed. Future already fini...
3,640,buildFromJson(): Error parsing channelInternal...
4,328,Request: GetChannelsRequest
5,222,ResourceEvaluationCount=: exceeds . Forbidden ...
6,135,onError(): com.android.volley.NoConnectionErro...
7,88,[SM.UrlParser ] Parsing DVB URL was not...
8,82,[SM.UrlParser ] Error at location: .\n
9,69,[CfgM.Configuration ] Access to unknown prope...


In [23]:
print("Top terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print
 

Top terms per cluster:
Cluster 0:  setresult  finished  already  future  result  failed  zygoteinit  getseachangeid  google  greenrobot
Cluster 1:  host  address  ch  unknownhostexception  associated  hostname  no  with  traxis  resolve
Cluster 2:  process  took  ms  sleep  appe  zygoteinit  handlemessage  getchannelsrequest  getseachangeid  google
Cluster 3:  channelinternalproviderdata  buildfromjson  parsing  error  zygoteinit  handlerthread  greenrobot  handlecallback  handlemessage  handler
Cluster 4:  environment  exceeds  in  production  forbidden  resourceevaluationcount  zygoteinit  handler  google  greenrobot
Cluster 5:  getchannelsrequest  request  zygoteinit  handlemessage  getseachangeid  google  greenrobot  handlecallback  handler  geoblock
Cluster 6:  location  at  urlparser  sm  error  handlecallback  getchannelsrequest  getseachangeid  google  greenrobot
Cluster 7:  xml  path  illegal  osal  file  inarisosalfileopen  sgrp_default  scanchannels  zygoteinit  handlemessag

In [24]:
cluster = 0
indices = [i for i, x in enumerate(labels) if x == cluster]
[corpus[i] for i in indices]

[u'setResult(): failed. Future already finished.',
 u'setResult(result): failed. Future already finished.',
 u'setResult(): failed. Future already finished.',
 u'setResult(): failed. Future already finished.',
 u'setResult(result): failed. Future already finished.',
 u'setResult(result): failed. Future already finished.',
 u'setResult(): failed. Future already finished.',
 u'setResult(): failed. Future already finished.',
 u'setResult(): failed. Future already finished.',
 u'setResult(): failed. Future already finished.',
 u'setResult(): failed. Future already finished.',
 u'setResult(): failed. Future already finished.',
 u'setResult(): failed. Future already finished.',
 u'setResult(): failed. Future already finished.',
 u'setResult(): failed. Future already finished.',
 u'setResult(): failed. Future already finished.',
 u'setResult(result): failed. Future already finished.',
 u'setResult(): failed. Future already finished.',
 u'setResult(): failed. Future already finished.',
 u'setR

In [25]:
date_cols = ['timestamp', 'timespan_start', 'timespan_end']
service = splunk_api.get_connection(do_remote=1)
splunk_time_format = "%m/%d/%Y:%H:%M:%S"

for index, row in df.iterrows():
    directory = "splunk_results"
    if not os.path.exists(directory):
        os.makedirs(directory)
    filename = os.path.join(directory, "saved_{}.csv".format(index))
    if os.path.exists(filename):
        print("skipping", filename)
        continue        

    splunk_query = 'search {mac_address} earliest={start_time} latest={end_time} (source="D:\\\\Logs\\\\TraxisService.log" OR source="E:\\\\Logs\\\\TraxisService.log")'.format(
        mac_address = row['mac'].replace(':', ''),
        start_time = row['timespan_start'].strftime(splunk_time_format),
        end_time = row['timespan_end'].strftime(splunk_time_format),
        )
    print(splunk_query)

    df = splunk_api.execute_query(service, splunk_query)
    print(df)

IOError: [Errno 2] No such file or directory: 'splunk_api.json'